### Silbentrennung [Bwinf44 2025](https://bwinf.de/bundeswettbewerb/44/) - J2

[Video]()



<img src='silbentrennung.png' width='250'>

#### Beispieldaten

Jede Datei enthält einen Text, den Lars nach Silben aufteilen möchte. 
Wenn Lars lediglich seine eigenen Regeln anwenden würde, wäre das Ergebnis für silben05.txt diese Aufteilung:

Se-i-n Va-te-r ist Ba-u-e-r und er-nte-t Ma-i-s.

In [139]:
# Anschauen der Beispieldaten 01-08
nr = '01'
print(f'Beispiel {nr}:')
f = open(f'beispieldaten/silben{nr}.txt')
print(f.read().strip())
f.close()

Beispiel 01:
Mein Name ist Lars und ich esse schrecklich gerne Sauerkraut.


#### Lösungsidee

#### Umsetzung

Zunächst definieren wir uns ein paar Variablen. Jedes Zeichen, was kein Buchstabe ist, ist ein Satzzeichen.


In [140]:
vokale = 'aeiouäöüAEIOUÄÖÜ'
konsonanten = 'bcdfghjklmnpqrstvwxyzBCDFGHJKLMNPQRSTVWXYZ'
buchstaben = vokale + konsonanten

Wir setzen jede Regel in einer eigenen Funktion um und überprüfen diese mit ein paar Beispielen.

Die Liste, die eine Regel zurückgibt, nennen wir Trennliste. 
Am Index k steht 

- 1 falls nach Zeichen am Index k getrennt werden darf
- -1 falls nach Zeichen  am Index k nicht getrennt werden darf
- 0 falls wir keine Information darüber haben, ob am Index k getrennt werden darf


##### Regel1

In [141]:
def regel1(wort):
    ''' Ein Wort kann zwischen zwei Konsonanten getrennt werden '''
    tmp = [0]*(len(wort))
    for i in range(0,len(wort) - 1):
        if wort[i] in konsonanten and wort[i + 1] in konsonanten:
            tmp[i] = 1
    return tmp

In [142]:
regel1('Sauerkraut')

[0, 0, 0, 0, 1, 1, 0, 0, 0, 0]

In [143]:
def trennung(wort,trennliste):
    ''' returns: die Trennung von wort an den 1er-Stellen der trennliste '''
    tmp = ''
    for i in range(len(wort)):
        if trennliste[i] == 1:
            tmp += wort[i]+'-'
        else:
            tmp +=  wort[i]
    return tmp

In [144]:
wort = 'Sauerkraut'
trennung(wort, regel1(wort))

'Sauer-k-raut'

In [159]:
nr = '01'
f = open(f'beispieldaten/silben{nr}.txt',encoding='utf-8')
satz = f.readline()
f.close()
for wort in satz.split():
    print(trennung(wort, regel1(wort)), end=' ')

Mein Name is-t Lar-s un-d ic-h es-se s-c-h-rec-k-lic-h ger-ne Sauer-k-raut. 

##### Regel 2

Bei Regel 2 müssen wir berücksichtigen, dass das letzte Zeichen in dem gesplitteten Satz ein Satzzeichen sein kann.

In [146]:
def regel2(wort):
    '''Erster und letzter Buchstabe eines Wortes dürfen nicht abgetrennt werden '''
    erster = 0
    while wort[erster] not in buchstaben and erster < len(wort):
        erster += 1
    letzter = len(wort) - 1
    while wort[letzter] not in buchstaben and letzter > 0:
        letzter -= 1

    tmp = [0]*(len(wort))
    tmp[erster] = -1
    tmp[letzter-1] = -1
    return tmp

In [147]:
regel2('Sauerkraut.')

[-1, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0]

In [148]:
regel2('schrecklich')

[-1, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0]

##### Regel 3

In [149]:
def regel3(wort):
    '''bei drei aufeinanderfolgenden Konsonanten kann nur nach dem ersten Konsonanten getrennt werden '''
    tmp = [0]*(len(wort))
    for i in range(1,len(wort) - 2):
        if tmp[i] != 0: continue
        if wort[i] in konsonanten and wort[i + 1] in konsonanten and wort[i + 2] in konsonanten:
            tmp[i] = 1
            tmp[i+1] = -1
            tmp[i+2] = -1
    return tmp

In [150]:
for wort in satz.split():
    print(trennung(wort, regel3(wort)), end=' ')

Mein Name ist Lars und ich esse sc-hrec-klich gerne Sauer-kraut. 

##### Regel 4

In [151]:
def regel4(wort):
    ''' Ein Wort kann nach einem Vokal getrennt werden, falls nicht zwei Konsonanten folgen '''
    tmp = [0]*(len(wort))
    for i in range(1,len(wort) - 2):
        if wort[i] in vokale:
            if wort[i + 1] in konsonanten and wort[i + 2] in konsonanten:
                tmp[i] = -1
            else:
                tmp[i] = 1            
    k = len(wort)-2
    if wort[k] in vokale and wort[k + 1] in buchstaben: 
        tmp[k] = 1

    return tmp

In [152]:
for wort in satz.split():
    print(trennung(wort, regel4(wort)), end=' ')

Me-i-n Na-me ist Lars und ich esse schrecklich gerne Sa-u-erkra-u-t. 

#### Alle Regeln zusammenfassen

In [154]:
wort = 'Sauerkraut.'
trennlisten = [regel4(wort), regel3(wort), regel2(wort), regel1(wort)]
trennlisten

[[0, 1, 1, -1, 0, 0, 0, 1, 1, 0, 0],
 [0, 0, 0, 0, 1, -1, -1, 0, 0, 0, 0],
 [-1, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0],
 [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]]

In [155]:
def trennliste_gesamt(trennlisten):
    n = len(trennlisten[0])
    tmp = [0]*n
    for i in range(n):
        for t in trennlisten:
            if t[i] == -1: break
            if t[i] == 1:
                tmp[i] = 1
                break
    return tmp

In [156]:
trennliste_gesamt(trennlisten)

[0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0]

In [157]:
for wort in satz.split():
    trennlisten = [regel4(wort), regel3(wort),regel2(wort), regel1(wort)]
    print(trennung(wort, trennliste_gesamt(trennlisten)), end=' ')

Me-i-n Na-me ist Lars und ich es-se sc-hrec-klich ger-ne Sa-u-er-kra-u-t. 

In [158]:
nr = '05'
f = open(f'beispieldaten/silben{nr}.txt',encoding='utf-8')
satz = f.readline()
for wort in satz.split():
    trennlisten = [regel4(wort), regel3(wort),regel2(wort), regel1(wort)]
    print(trennung(wort, trennliste_gesamt(trennlisten)), end=' ')

Se-i-n Va-te-r ist Ba-u-e-r und er-nte-t Ma-i-s. 

#### Das ganze Programm

In [166]:
def regel1(wort):
    ''' Ein Wort kann zwischen zwei Konsonanten getrennt werden '''
    tmp = [0]*(len(wort))
    for i in range(0,len(wort) - 1):
        if wort[i] in konsonanten and wort[i + 1] in konsonanten:
            tmp[i] = 1
    return tmp

def regel2(wort):
    '''Erster und letzter Buchstabe eines Wortes dürfen nicht abgetrennt werden '''
    erster = 0
    while wort[erster] not in buchstaben and erster < len(wort):
        erster += 1
    letzter = len(wort) - 1
    while wort[letzter] not in buchstaben and letzter > 0:
        letzter -= 1

    tmp = [0]*(len(wort))
    tmp[erster] = -1
    tmp[letzter-1] = -1
    return tmp

def regel3(wort):
    '''bei drei aufeinanderfolgenden Konsonanten kann nur nach dem ersten Konsonanten getrennt werden '''
    tmp = [0]*(len(wort))
    for i in range(1,len(wort) - 2):
        if tmp[i] != 0: continue
        if wort[i] in konsonanten and wort[i + 1] in konsonanten and wort[i + 2] in konsonanten:
            tmp[i] = 1
            tmp[i+1] = -1
            tmp[i+2] = -1
    return tmp

def regel4(wort):
    ''' Ein Wort kann nach einem Vokal getrennt werden, falls nicht zwei Konsonanten folgen '''
    tmp = [0]*(len(wort))
    for i in range(1,len(wort) - 2):
        if wort[i] in vokale:
            if wort[i + 1] in konsonanten and wort[i + 2] in konsonanten:
                tmp[i] = -1
            else:
                tmp[i] = 1            
    k = len(wort)-2
    if wort[k] in vokale and wort[k + 1] in buchstaben: 
        tmp[k] = 1

    return tmp

def trennliste_gesamt(trennlisten):
    n = len(trennlisten[0])
    tmp = [0]*n
    for i in range(n):
        for t in trennlisten:
            if t[i] == -1: break
            if t[i] == 1:
                tmp[i] = 1
                break
    return tmp

def trennung(wort,trennliste):
    ''' returns: die Trennung von wort an den 1er-Stellen der trennliste '''
    tmp = ''
    for i in range(len(wort)):
        if trennliste[i] == 1:
            tmp += wort[i]+'-'
        else:
            tmp +=  wort[i]
    return tmp

vkale = 'aeiouäöüAEIOUÄÖÜ'
konsonanten = 'bcdfghjklmnpqrstvwxyzBCDFGHJKLMNPQRSTVWXYZ'
buchstaben = vokale + konsonanten

nr = '08'
f = open(f'beispieldaten/silben{nr}.txt',encoding='utf-8')
satz = f.readline()
f.close()

for wort in satz.split():
    trennlisten = [regel4(wort), regel3(wort),regel2(wort), regel1(wort)]
    print(trennung(wort, trennliste_gesamt(trennlisten)), end=' ')

Fre-u-de, sc-hö-ne-r Göt-ter-fun-ke-n, Toc-hte-r au-s Eli-si-u-m, Wi-r bet-re-te-n fe-u-er-trun-ke-n, Him-mlis-che, de-i-n He-i-lig-thu-m. De-i-ne Za-u-be-r bin-de-n wi-e-de-r, Wa-s di-e Mo-de st-reng get-he-ilt, Al-le Men-sche-n wer-de-n Brü-de-r, Wo de-i-n san-fte-r Flü-ge-l we-ilt. 